# RF matching network

A series of single stub impedance matching networks were assembled using 50 Ω coaxial cables and characterized around 120 MHz.
The load was a termination with nominal value of 150 Ω, but, as assembled, it was closer to (134 - j40) Ω.

All data from the spectrum analyzer are stored in CSV files.

**Objective**: explore, filter and present the data.

In [ ]:
# Import necessary modules
import numpy
from matplotlib import pyplot

# Only in Jupyter Notebooks
%matplotlib notebook

In [ ]:
%cat matching/50ohm.csv

The header information (44 lines) must be skipped by `numpy.loadtxt`:

In [ ]:
data = numpy.loadtxt('matching/50ohm.csv', delimiter=',', skiprows=44)
data.shape

Before measuring the matching networks, the bridge setup was characterized with a matched impedance (50 Ω), the load (150 Ω), a short, and an open terminations.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

for fname in ['50ohm', '150ohm', 'open', 'short']:
    data = numpy.loadtxt('matching/{}.csv'.format(fname),
                         delimiter=',', skiprows=44)
    ax.plot(data[:, 0] * 1e-6, data[:, 1])

ax.legend([r'$50\,\Omega$', r'$150\,\Omega$', 'Open', 'Short'])
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Power (dBm)')
ax.grid()

Open and short terminations present relatively similar signal levels, as expected.
Because they represent total reflection at the load, they can be used as reference to  eliminate the effects of losses in the cables, connections and the bridge used in the measurement setup.

Because both curves are relatively constant, the average value is sufficient as a general reference value for all frequencies.

In [ ]:
short = numpy.loadtxt('matching/short.csv', delimiter=',',
                      skiprows=44, usecols=1)
short.shape

In [ ]:
reference = short.mean()
reference

The data file names include the lenghts (in centimeters) of the load (`d`) and short (`l`) stubs in each configuration.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

for d in range(23, 28):
    for l in range(16, 21):
        data = numpy.loadtxt('matching/d{}-l{}.csv'.format(d, l),
                             delimiter=',', skiprows=44)
        ax.plot(data[:, 0] * 1e-6, data[:, 1] - reference,
                label='d = {} cm, l = {} cm'.format(d, l))

ax.legend()
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Reflection (dB)')
ax.grid()

Even without the legend, this plot is heavily polluted.
It is difficult to extract any usefull information from it.

Considering that the data is 3-dimensional (frequency, `d` and `l` are independent variables), there are better ways to filter out and display usefull information from the measurements.

Since all measurements contain were taken at the same frequency values, it can be stored in a 3-d array indexed by (frequency, `l` value, `d` value).

In [ ]:
d = numpy.arange(23, 28)
l = numpy.arange(16, 21)
for dd in d:
    for ll in l:
        # In the first iteration the freq array is extracted and the 3-d array initialized
        if dd == d[0] and ll == l[0]:
            data = numpy.loadtxt('matching/d{}-l{}.csv'.format(dd, ll),
                             delimiter=',', skiprows=44)
            freq = data[:, 0] * 1e-6
            refl = numpy.empty((len(freq), 5, 5))
            refl[:, 0, 0] = data[:, 1]
        else:
            refl[:, ll - 16, dd - 23] = numpy.loadtxt(
                'matching/d{}-l{}.csv'.format(dd, ll),
                delimiter=',', skiprows=44, usecols=1)
refl -= reference

print(freq.shape, l.shape, d.shape, refl.shape)

The matching performance can be chcecked at the target frequency of 120 MHz.

The data is cut accross the frequency axis at the closest index (`freq[i]`) to 120 MHz.

In [ ]:
i = numpy.argmin(numpy.abs(freq - 120))
i, freq[i]

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.imshow(refl[i, :, :], origin='lower')

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.imshow(refl[i, :, :], origin='lower', cmap='magma',
                extent=(d[0]-0.5, d[-1]+0.5, l[0]-0.5, l[-1]+0.5))

pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

Alternatively, contours can be used when they do not introduce artifacts.

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d, l, refl[i, :, :])
pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contour(d, l, refl[i, :, :])
ax.clabel(cax, fmt='%.1f', fontsize='small')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

The data also shows the miminal reflection for each configuration across frequencies.

For example, for `d` = 23 cm and `l` = 16 cm, the minimum is:

In [ ]:
refl[:, 0, 0].min()

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d, l, refl.min(axis=0))
pyplot.colorbar(cax, label='Reflection (dB)')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

At which frequencies do the minima occur?

In [ ]:
refl.argmin(axis=0)

In [ ]:
refl[841, 0, 0] == refl[:, 0, 0].min()

In [ ]:
freq[refl.argmin(axis=0)]

In [ ]:
fig, ax = pyplot.subplots(1, 1)

cax = ax.contour(d, l, freq[refl.argmin(axis=0)])
ax.clabel(cax, fmt='%.0f MHz', fontsize='small')

ax.set_xlabel('d (cm)')
ax.set_ylabel('l (cm)')

## Exercise

Draw the plots of minimal reflection and frequency of minimal reflection for each network in a single `AxesSubplot` in such a way that both are presented to the reader.

### Bonus

Format the plot so that it can be printed in black and white without loss of information.

## Theoretical model

In [ ]:
f, l, d = numpy.mgrid[100:140:81j, 16:21:121j, 23:28:121j]
lda = 2.0e4 / f

zL = (134 - 40j) / 50.0
gL = (zL - 1) / (zL + 1)
gd = gL * numpy.exp(-numpy.pi*4j*d/lda)
zd = (1 + gd) / (1 - gd)

zs = 0
gs = (zs - 1) / (zs + 1)
gl = gs * numpy.exp(-numpy.pi*4j*l/lda)
zl = (1 + gl) / (1 - gl)

zin = 1 / (1/zd + 1/zl)
gin = (zin - 1) / (zin + 1)
refl = 20*numpy.log10(numpy.abs(gin))

fig, ax = pyplot.subplots(1, 1)

cax = ax.contourf(d[0,0,:], l[0,:,0], refl.min(axis=0), cmap='gray',
                 levels=numpy.arange(-40, -19, 2.5), extend='min')
pyplot.colorbar(cax, label='Reflection (dB)')

cax = ax.contour(d[0,0,:], l[0,:,0], f[:,0,0][refl.argmin(axis=0)],
                 cmap='winter')
ax.clabel(cax, fmt='%.0f MHz', fontsize='small')

ax.set_xlabel(r"$d$ (cm)")
ax.set_ylabel(r"$\ell$ (cm)")

ax.set_aspect(1)